In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DBCONN")

In [3]:
pip install "psycopg[binary]" 


   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 28.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install psycopg

Note: you may need to restart the kernel to use updated packages.


In [26]:
import psycopg

# establish a connection to the database
conn = psycopg.connect(dbconn)

# or use the connection string directly
# create a cursor object
cur = conn.cursor()

AttributeError: 'NoneType' object has no attribute 'encode'

In [ ]:
# Create a table for the stock market data
def create_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS api_data(
                date TIMESTAMP PRIMARY KEY,
                open FLOAT,
                high FLOAT,
                low FLOAT,
                close FLOAT,
                volume FLOAT
                            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_table()


In [ ]:
# Create a table for the article data
def create_article_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS article_data(
                date TIMESTAMP PRIMARY KEY,
                title TEXT,
                author TEXT,
                link TEXT
            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_article_table()

In [ ]:
import pandas as pd

btc_df = pd.read_csv('btc_prices.csv')
btc_df.head()

In [20]:
# Insert stock price data to the supabase table

from datetime import datetime
import psycopg
import pandas as pd

def insert_stock_data_from_df(df):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # Convert date to datetime if needed
        if isinstance(row['date'], str):
            date_obj = datetime.strptime(row['date'], "%Y-%m-%d")
        else:
            date_obj = pd.to_datetime(row['date'])

        cur.execute(
            '''
            INSERT INTO api_data (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            ''',
            (date_obj, row['open'], row['high'], row['low'], row['close'], row['volume'])
        )

    conn.commit()
    cur.close()
    conn.close()

# Then call it like this:
insert_stock_data_from_df(btc_df)


NameError: name 'btc_df' is not defined

## Articles Database

In [21]:
import pandas as pd
article_df = pd.read_csv('u_today_articles.csv')
article_df.head()

,date,title,author,link
0,"Jun 3, 2025 - 8:26","Tether Moves $1,107,371,328 Bitcoin, CEO Break...",Gamza Khanzadaev,https://u.today/tether-moves-1107371328-bitcoi...
1,"Jun 3, 2025 - 7:29",Gold Copying Bitcoin’s Breakout Trajectory: Le...,Yuri Molchan,https://u.today/gold-copying-bitcoins-breakout...
2,"Jun 3, 2025 - 6:04",Bitcoin ETFs: SEC Commissioner Says Agency to ...,Alex Dovbnya,https://u.today/bitcoin-etfs-sec-commissioner-...
3,"Jun 3, 2025 - 1:13",$5.86 Billion In Bitcoin Flows Into ETFs In May,Caroline Amosun,https://u.today/586-billion-in-bitcoin-flows-i...
4,"Jun 2, 2025 - 23:43",Strategy to Boost Bitcoin (BTC) Acquisitions w...,Paul Adedoyin,https://u.today/strategy-to-boost-bitcoin-btc-...


In [ ]:
import psycopg
from datetime import datetime

def insert_article_data_from_df(df):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # Convert date to datetime if needed
        if isinstance(row['date'], str):
            try:
                date_obj = datetime.strptime(row['date'], "%b %d, %Y - %H:%M")
            except ValueError:
                # Try alternative format if needed
                date_obj = pd.to_datetime(row['date'])
        else:
            date_obj = pd.to_datetime(row['date'])

        cur.execute(
            '''
            INSERT INTO article_data (date, title, author, link)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            ''',
            (date_obj, row['title'], row['author'], row['link'])
        )

    conn.commit()
    cur.close()
    conn.close()

insert_article_data_from_df(article_df)

AttributeError: 'NoneType' object has no attribute 'encode'

In [16]:
# Update the article data with new articles from June 4th to June 12th, 2025
from datetime import datetime
import pandas as pd
import psycopg

def update_article_data_from_df(df):
    # Parse dates safely
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    
    # Filter for June 4–12, 2025
    start_date = pd.Timestamp('2025-06-04')
    end_date = pd.Timestamp('2025-06-12 23:59:59')  # Include full day of June 12
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # Database insert/update
    with psycopg.connect(dbconn) as conn:
        with conn.cursor() as cur:
            for _, row in df.iterrows():
                cur.execute(
                    '''
                    INSERT INTO article_data (date, title, author, link)
                    VALUES (%s, %s, %s, %s)
                    ON CONFLICT (date) DO UPDATE SET
                        title = EXCLUDED.title,
                        author = EXCLUDED.author,
                        link = EXCLUDED.link;
                    ''',
                    (row['date'], row['title'], row['author'], row['link'])
                )
        conn.commit()
        
  

## Add missing articles to Supabase

In [31]:
# Push filtered articles (df_filtered) to Supabase

if not df_filtered.empty:
    # Add sslmode='require' to the connection
    with psycopg.connect(dbconn, sslmode='require') as conn:
        with conn.cursor() as cur:
            for _, row in df_filtered.iterrows():
                cur.execute(
                    '''
                    INSERT INTO article_data (date, title, author, link)
                    VALUES (%s, %s, %s, %s)
                    ON CONFLICT (date) DO NOTHING;
                    ''',
                    (row['date'], row['title'], row['author'], row['link'])
                )
        conn.commit()
    print(f"Pushed {len(df_filtered)} articles from June 4–12, 2025 to Supabase.")
else:
    print("No articles found for June 4–12, 2025.")


OperationalError: connection failed: connection to server at "52.59.152.35", port 5432 failed: received invalid response to GSSAPI negotiation: S

In [2]:
import os
from dotenv import load_dotenv
import psycopg

load_dotenv()
dbconn = os.getenv("DB_CONN")

try:
    with psycopg.connect(dbconn) as conn:
        print("✅ Connected to Supabase PostgreSQL!")
except Exception as e:
    print("❌ Connection failed:", e)


❌ Connection failed: 'NoneType' object has no attribute 'encode'
